In [31]:
import pandas as pd
import gzip
import re

In [32]:
# def read_tbl():
#     df = pd.read_excel("../../../../data/NM/23-6-26/copy_of_ipra_request_po16267_fulldata__sheets-20231220144255.xlsx")
#     return df 

# df = read_tbl()

# # Select the columns needed for processing
# person_columns = [
#     'Person Certification #', 'Person First Name', 'Person Middle Name',
#     'Person Last Name', 'Person Suffix', 'Person Gender', 'Person Date of Birth',
#     'Year of Birth', 'Person EEOC Category'
# ]

# # Define the base names for employment columns
# employment_base_columns = [
#     'Employment Start Date', 'Employment End Date', 'Employment Appointment Type*',
#     'Employment Employment Type*', 'Employment Title/Rank (Current)',
#     'Employment Status', 'Is Primary Employment', 'Employing Organization Name',
#     'Employing Organization Agency Type'
# ]

# # Function to get all employment-related columns
# def get_employment_columns(df):
#     employment_columns = []
#     for col in df.columns:
#         if col.startswith('Employment') or col.startswith('Employing') or col == 'Is Primary Employment':
#             employment_columns.append(col)
#     return employment_columns

# # Get all employment-related columns
# employment_columns = get_employment_columns(df)

# # Combine person columns and employment columns
# columns_to_keep = person_columns + employment_columns

# # Filter the dataframe to keep only the relevant columns
# filtered_df = df[columns_to_keep]

# # Create an empty DataFrame to store the cleaned data
# cleaned_df = pd.DataFrame()

# # Function to get the base column name
# def get_base_column(col):
#     for base in employment_base_columns:
#         if col.startswith(base):
#             return base
#     return col

# # Function to extract number from column name
# def extract_number(col):
#     match = re.search(r'\d+$', col)
#     return int(match.group()) if match else 0

# # Group employment columns
# employment_groups = {}
# for col in employment_columns:
#     base_col = get_base_column(col)
#     if base_col not in employment_groups:
#         employment_groups[base_col] = []
#     employment_groups[base_col].append(col)

# # Sort columns in each group
# for base_col in employment_groups:
#     employment_groups[base_col].sort(key=extract_number)

# # Determine the maximum number of employment stints
# max_stints = max(len(group) for group in employment_groups.values())

# # Iterate through the rows and extract each employment stint for each person
# for idx, row in filtered_df.iterrows():
#     person_info = row[person_columns]
    
#     for i in range(max_stints):
#         employment_info = {}
#         for base_col in employment_base_columns:
#             if base_col in employment_groups and i < len(employment_groups[base_col]):
#                 col = employment_groups[base_col][i]
#                 employment_info[base_col] = row[col]
#             else:
#                 employment_info[base_col] = pd.NA
        
#         # Check if at least one column in this set is not null
#         if any(pd.notna(value) for value in employment_info.values()):
#             # Combine person info and employment info
#             combined_info = pd.concat([person_info, pd.Series(employment_info)])
#             cleaned_df = pd.concat([cleaned_df, combined_info.to_frame().T], ignore_index=True)

# # Reset the index of the cleaned dataframe
# cleaned_df.reset_index(drop=True, inplace=True)

# cleaned_df.to_csv("../data/output/NM.csv")

In [33]:
def clean_dates(df):
    df.loc[:, "start_date"] = df.start_date.str.replace(r"(\w+):(\w+):(\w+)", "", regex=True)
    df.loc[:, "end_date"] = df.end_date.str.replace(r"(\w+):(\w+):(\w+)", "", regex=True)
    return df 

def clean_race(df):
    df.loc[:, "race"] = df.race.str.replace(r"Caucasian", "white", regex=False)
    return df 

def clean_agency(df):
    df.loc[:, "agency_name"] = (df
                           .agency_name
                           .str.lower()
                           .str.strip()
                           .str.replace(r"dept\.", "department", regex=True)
                           .str.replace(r"&", "and", regex=False)
                           .str.replace(r"\bpd\b", "police department", regex=True)
                           .str.replace(r"(\w+)  (\w+)", r"\1 \2", regex=True)
                           .str.replace(r"mex\.", "mexico", regex=True)

    )
    df = df[~((df.agency_name.str.contains("fire")))]
    df.loc[:, "agency_name"] = df.agency_name.str.title()
    return df

def read_tbl():
    df = pd.read_csv("../data/output/NM.csv")

    df = df.rename(columns={"Person Certification #": "person_nbr", 
                           "Person First Name": "first_name", 
                           "Person Middle Name": "middle_name", 
                           "Person Last Name": "last_name", 
                           "Person Suffix": "suffix",
                           "Person Gender": "sex", 
                           "Person EEOC Category": "race",
                           "Year of Birth": "year_of_birth", 
                           "Employment Start Date": "start_date", 
                           "Employment End Date": "end_date", 
                           "Employment Employment Type*": "employment_type", 
                           "Employment Status": "employment_status", 
                           "Employing Organization Name": "agency_name", 
                           "Employing Organization Agency Type": "agency_type", 
                           "Employment Title/Rank (Current)": "rank"})
    
    df = df.fillna("")
    
    df = df[["person_nbr", "first_name", "middle_name", "last_name", "suffix", "sex", 
             "year_of_birth", "race", "start_date", "end_date", "employment_type", 
             "rank", "employment_status", "agency_name", "agency_type" ]]
    return df

df = read_tbl()


df = df.pipe(clean_dates).pipe(clean_race).pipe(clean_agency)


df.to_csv("../data/output/new-mexico-preprocessed.csv", index=False)

In [34]:
def clean_special_characters(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].apply(lambda x: ''.join(char for char in str(x) if ord(char) >= 32 or char == '\n'))
    return df

def trim_whitespace(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.strip()
    return df

def standardize_dates(df):
    date_columns = ['start_date', 'end_date']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')
    return df

def handle_newlines(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.replace('\n', ' ')
    return df

def ensure_utf8(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].apply(lambda x: x.encode('utf-8', errors='ignore').decode('utf-8') if isinstance(x, str) else x)
    return df

def post_process(df):
    return (df
            .pipe(clean_special_characters)
            .pipe(trim_whitespace)
            .pipe(standardize_dates)
            .pipe(handle_newlines)
            .pipe(ensure_utf8)
           )

df = post_process(df)


df = df.drop_duplicates(subset=["person_nbr", "agency_name"])

df = df.sort_values("person_nbr")
df

# df.loc[:, "full_name"] = df.first_name.astype(str).fillna("") + " " + df.last_name.astype(str).fillna("") 

# # with gzip.open("../data/output/new mexico-processed.csv.gz", 'wt') as f:
# #     df.to_csv(f, index=False)

,person_nbr,first_name,middle_name,last_name,suffix,sex,year_of_birth,race,start_date,end_date,employment_type,rank,employment_status,agency_name,agency_type
4560,00-0001-P,Joshua,D,Anderson,,Male,1977,white,2000-05-22,2019-10-31,Full Time,Sergeant,Retired,Bernalillo Police Department,Law Enforcement Agency
4561,00-0001-P,Joshua,D,Anderson,,Male,1977,white,1999-07-26,2000-04-17,,Patrol Officer,Resigned,Farmington Police Department,Law Enforcement Agency
16413,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,1999-07-18,2001-06-23,,Patrol Officer,Resigned,Aztec Police Department,Law Enforcement Agency
16414,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,2014-08-25,2016-08-01,,[Missing Title/Rank],Resigned,Farmington Police Department,Law Enforcement Agency
16415,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,2001-07-09,2014-08-25,,Deputy Sheriff,Resigned,San Juan County Sheriff'S Office,Law Enforcement Agency
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10046,99-0435-P,Ronald,,Williams,,Male,1974,Native American,1999-07-19,2010-02-22,,Patrol Officer,Resigned,Navajo Department Of Public Safety,Law Enforcement Agency
9907,99-0436-P,Gilbert,,Yazzie,,Male,1971,Native American,1999-08-04,NaN,Full Time,Patrol Officer,Active,Navajo Department Of Public Safety,Law Enforcement Agency
10869,99-0438-P,Harry,Kyle,Bayless,,Male,1938,white,2009-10-03,NaN,Part Time,Patrol Officer,Active,New Mexico Military Institute Police Dep,Law Enforcement Agency
10870,99-0438-P,Harry,Kyle,Bayless,,Male,1938,white,1999-12-03,2000-09-29,,Patrol Officer,Resigned,Capitan Police Department,Law Enforcement Agency


In [35]:
# df.to_csv("../data/output/new-mexico_index.csv", index=False)